In [1]:
import pandas as pd

# Data cleaning

## Movie_titles cleaning

In [2]:
def manual_separation(bad_line):
    right_split = bad_line[:-2] + [",".join(bad_line[-2:])] # All the "bad lines" where all coming from the same last column that was containing ","
    return right_split


movie_titles_df = pd.read_csv(
        "src/movie_titles.csv", 
        header=None,
        encoding="ISO-8859-1",
        names = ['Movie_Id', 'Year', 'Title'], 
        # index_col=['movie_id'],
        on_bad_lines=manual_separation,
        engine="python",
    ) 

C:\Users\renau\AppData\Local\Temp\ipykernel_212756\6465067.py:6: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  movie_titles_df = pd.read_csv(


In [3]:
print(movie_titles_df.shape)

display(movie_titles_df.head())

display(movie_titles_df.describe())

print("Percentage of missing values\n")
print(movie_titles_df.isnull().sum()/len(movie_titles_df)*100)

(17770, 3)


,Movie_Id,Year,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


,Movie_Id,Year
count,17770.000000,17763.000000
mean,8885.500000,1990.263694
std,5129.901477,16.507891
min,1.000000,1896.000000
25%,4443.250000,1985.000000
50%,8885.500000,1997.000000
75%,13327.750000,2002.000000
max,17770.000000,2005.000000


Percentage of missing values

Movie_Id    0.000000
Year        0.039392
Title       0.000000
dtype: float64


## combined_data cleaning

In [4]:
#Import users data files
df_users_1 = pd.read_csv('src/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df_users_2 = pd.read_csv('src/combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df_users_3 = pd.read_csv('src/combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df_users_4 = pd.read_csv('src/combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

In [5]:
#concat into one file
df_users = pd.concat([df_users_1, df_users_2, df_users_3, df_users_4], axis=0)
df_users = df_users.reset_index(drop=True)

The data structure is not clean :

![alt text](img/screnshot_1.png)

It is necessary to get the movie ID as a column.

In [6]:
#Get the movie_id from the cust_id column, and create a 'Movie_Id' column
df_users['Movie_Id'] = df_users[df_users['Rating'].isna()]['Cust_Id'].str.replace(':', '')
df_users['Movie_Id'].fillna(method='ffill', inplace=True)
df_users.dropna(subset='Rating', inplace=True)

C:\Users\renau\AppData\Local\Temp\ipykernel_212756\3631577870.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_users['Movie_Id'].fillna(method='ffill', inplace=True)
C:\Users\renau\AppData\Local\Temp\ipykernel_212756\3631577870.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_users['Movie_Id'].fillna(method='ffill', inplace=True)


In [7]:
#some cleaning on columns type
df_users['Movie_Id']=df_users['Movie_Id'].astype(int)
df_users['Rating']=df_users['Rating'].astype(int)
df_users['Cust_Id']=df_users['Cust_Id'].astype(int)


In [8]:
#Merge between users data and movie title.
main_df = pd.merge(df_users, movie_titles_df, on='Movie_Id', how='left')

In [9]:
# Drop missing values in 'Year' column 
len_before_drop = len(main_df)
main_df = main_df.dropna()
print(f"Dropped {len_before_drop - len(main_df)} rows")

Dropped 965 rows


In [10]:
#convert 'Year' to int
main_df['Year'] = main_df['Year'].astype(int)

The dataset contains over **100 million rows**. It is necessary to filter part of the data to be able to use it.  
The dataset will be cleaned according to several conditions:
- Movies with too few reviews
- Users who gave too few reviews
- Movies rated under 4
- Movies released before 2000

For that, it is necessary to determines both thesholds

In [37]:
#compute movie_min_reviews
f = ['count','mean']

df_movie_summary = main_df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.75),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index
print('Movie minimum times of review: {}'.format(movie_benchmark))

Movie minimum times of review: 2670.0


In [38]:
#compute users_min_reviews
df_cust_summary = main_df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.75),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index
print('Customer minimum times of review: {}'.format(cust_benchmark))

Customer minimum times of review: 259.0


In [39]:
#compute the drop list of movies under 4 rating
df_4rated_movies = main_df.groupby('Movie_Id')['Rating'].agg(['mean'])
df_4rated_movies.index = df_4rated_movies.index.map(int)
drop_ratedmovie_list = df_4rated_movies[df_4rated_movies['mean'] < 4].index

In [64]:
print(f"raw dataset shape : {main_df.shape}")
filtered_df = main_df[~main_df['Movie_Id'].isin(drop_movie_list)]
filtered_df = filtered_df[~main_df['Movie_Id'].isin(drop_ratedmovie_list)]
filtered_df = filtered_df.loc[filtered_df['Year'] > 2000]
filtered_df = filtered_df[~main_df['Cust_Id'].isin(drop_cust_list)]
print(f"filtered dataset shape : {filtered_df.shape}")

raw dataset shape : (100479542, 5)


C:\Users\renau\AppData\Local\Temp\ipykernel_212756\3215412134.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = filtered_df[~main_df['Movie_Id'].isin(drop_ratedmovie_list)]
C:\Users\renau\AppData\Local\Temp\ipykernel_212756\3215412134.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = filtered_df[~main_df['Cust_Id'].isin(drop_cust_list)]


filtered dataset shape : (2717442, 5)


In [65]:
filtered_df.to_csv('src/filtered_data.csv', index=False)

In [73]:
#export movie title list for filtering Jedha API results
list_title = pd.DataFrame(filtered_df['Title'].unique(), columns=['Title'])
list_title.to_csv('src/list_title.csv', index=False)